In [1]:
import os
import twitter

from dotenv import load_dotenv

In [2]:
load_dotenv('../.env')

True

In [3]:
# consumer_key =  os.getenv("TWITTER_CONSUMER_KEY")
# consumer_secret = os.getenv("TWITTER_CONSUMER_SECRET")
# access_token_key = os.getenv("TWITTER_ACCESS_TOKEN_KEY")
# access_token_secret = os.getenv("TWITTER_ACCESS_TOKEN_SECRET")



# api = twitter.Api(consumer_key=consumer_key,
#                   consumer_secret=consumer_secret,
#                   access_token_key=access_token_key,
#                   access_token_secret=access_token_secret)
# print(api.VerifyCredentials())

- https://developer.twitter.com/en/docs/twitter-api/tweets/search/api-reference/get-tweets-search-all
- https://www.postman.com/twitter/workspace/twitter-s-public-workspace/collection/9956214-784efcda-ed4c-4491-a4c0-a26470a67400?ctx=documentation
- https://www.postman.com/twitter/workspace/twitter-s-public-workspace/request/9956214-e8d36c1b-9f1b-42b3-be14-cd624ffd038a

- query: 질의 내용, https://developer.twitter.com/en/docs/twitter-api/tweets/counts/integrate/build-a-query
- start_time: 시작시간 "YYYY-MM-DDTHH:mm:ssZ"
- end_time: 끝시간 "YYYY-MM-DDTHH:mm:ssZ"

- max_results: 최대 응답 개수 - 500 이하, 기본값 - 10

- expansions
    - tweet.fields
        - organic_metrics: 
        ```
        "organic_metrics": {
             "impression_count": 3880,
             "like_count": 8,
             "reply_count": 0,
             "retweet_count": 4
             "url_link_clicks": 3
             "user_profile_clicks": 2
            }
        ```
        
        attachments,author_id,context_annotations,conversation_id,created_at,entities,geo,id,in_reply_to_user_id,lang,possibly_sensitive,public_metrics,referenced_tweets,reply_settings,source,text,withheld
        - https://developer.twitter.com/en/docs/twitter-api/data-dictionary/object-model/tweet

- since_id: 사용자 아이디 관련
- until_id: 사용자 아이디 관련

In [34]:
import requests
import os
import json

# To set your environment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'
bearer_token = os.getenv("TWITTER_BEARER")
# bearer_token


In [15]:
url = "https://api.twitter.com/2/tweets/search/all?query=btc&start_time=2022-07-21T22:22:22Z&end_time=2022-07-21T22:22:32Z&tweet.fields=id,text,created_at,public_metrics"
headers = {
        "Authorization": "Bearer {}".format(bearer_token), 
        "User-Agent": "v2SpacesSearchPython"
    }

response = requests.request("GET", url, headers=headers)

                    

In [16]:
response

<Response [403]>

In [27]:

search_url = "https://api.twitter.com/2/tweets/search/all"

# Optional params: start_time,end_time,since_id,until_id,max_results,next_token,
# expansions,tweet.fields,media.fields,poll.fields,place.fields,user.fields
query_params = {'query': '(from:twitterdev -is:retweet) OR #twitterdev','tweet.fields': 'author_id'}


def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2FullArchiveSearchPython"
    return r


def connect_to_endpoint(url, params):
    response = requests.request("GET", search_url, auth=bearer_oauth, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()


#def main():
json_response = connect_to_endpoint(search_url, query_params)
print(json.dumps(json_response, indent=4, sort_keys=True))


403


Exception: (403, '{"client_id":"24929536","detail":"When authenticating requests to the Twitter API v2 endpoints, you must use keys and tokens from a Twitter developer App that is attached to a Project. You can create a project via the developer portal.","registration_url":"https://developer.twitter.com/en/docs/projects/overview","title":"Client Forbidden","required_enrollment":"Standard Basic","reason":"client-not-enrolled","type":"https://api.twitter.com/2/problems/client-forbidden"}')

- https://github.com/twitterdev/Twitter-API-v2-sample-code/issues/83

- https://github.com/jdfoote/Intro-to-Programming-and-Data-Science/blob/fall2021/extra_topics/twitter_v2_example.ipynb

In [29]:
import tweepy
# from twitter_authentication import bearer_token
import time
import pandas as pd

In [32]:
client = tweepy.Client(bearer_token, wait_on_rate_limit=True)

In [33]:
hoax_tweets = []
for response in tweepy.Paginator(client.search_all_tweets, 
                                 query = 'COVID hoax -is:retweet lang:en',
                                 user_fields = ['username', 'public_metrics', 'description', 'location'],
                                 tweet_fields = ['created_at', 'geo', 'public_metrics', 'text'],
                                 expansions = 'author_id',
                                 start_time = '2021-01-20T00:00:00Z',
                                 end_time = '2021-01-21T00:00:00Z',
                              max_results=500):
    time.sleep(1)
    hoax_tweets.append(response)

Forbidden: 403 Forbidden
When authenticating requests to the Twitter API v2 endpoints, you must use keys and tokens from a Twitter developer App that is attached to a Project. You can create a project via the developer portal.

# The full archive search endpoint in v2 is only available with academic access.

- https://developer.twitter.com/en/products/twitter-api/academic-research